---
# Feb 28, 2020 Metric 3 HW1 BLP (1995)
---
* Name: Jikhan Jeong
* Logistics:  Due to the limited capcity and low understanding, **the most of code is from online resources and modified by me**.
* Env: All the code and answer are in one Jupyter notebook file
* Paper: Berry, Steven, James Levinsohn, and Ariel Pakes. "Automobile prices in market equilibrium." Econometrica: Journal of the Econometric Society (1995): 841-890.
* Data: cardata.txt (https://kohei-kawaguchi.github.io/EmpiricalIO/assignment4.html, data source)
* Ref: https://mark-ponder.com/tutorials/static-discrete-choice-models/random-coefficients-blp/ (Julia)
* Ref: https://kohei-kawaguchi.github.io/EmpiricalIO/assignment4.html (R)
* Ref: https://jeffgortmaker.com/files/pyblp.pdf (Python Package)
* Ref: https://hktosun.github.io/coursework/blp.html (R & Python)
* Ref: http://www.ivan-li.com/code/blp_1995 (Python)
* Ref: https://python.quantecon.org/ols.html (2sls, Python statsmodel package)
---

# 0. Load data 
---

In [324]:
import os
os.chdir('/data/cahnrs/jikhan.jeong/metric3_2020')
print(os.getcwd())

/data/cahnrs/jikhan.jeong/metric3_2020


---
### 0-1: Matlab Data Load
* Data structure equires more datawork. so I gave up to use it
* Using alterative txt format dataset
---

In [325]:
import mat4py as mtp
raw = mtp.loadmat('BLP_data.mat')
raw = pd.DataFrame.from_dict(raw, orient ='index').T
raw.head(1)
# Give up to use this dataset.
# Ref: https://stackoverflow.com/questions/40828797/writing-python-numpy-arrays-to-mat-file (mat4py)

,model_id,own_dummies,id,product,const,mpd,air,mpg,trend,space,hpwt,cdindex,cdid,outshr,firmid,share,price
0,[23],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",[129],[1],[1],[1.8881456043956044],[0],[1.6969999999999998],[0],[1.1502000000000001],[0.5289968652037618],[92],[1],[0.8801062901193832],[15],[0.0010512928190053391],[4.935802469135802]


# Loading Package and Pre-defined functions

In [327]:
import numpy as np
import pandas as pd
from numba import jit
from numpy.linalg import inv # inverse matrix for 2sls coefficients
import matplotlib.pyplot as plt
from scipy.optimize import minimize # optimization BFGS
from scipy import stats # to derive p-value
import pyblp # used for replication study and not used in here

np.set_printoptions(linewidth=np.inf) # For display in one row : ref: https://stackoverflow.com/questions/56204072/numpy-print-matrix-one-row-one-line
np.set_printoptions(threshold=np.inf) # For display in all column : ref: https://stackoverflow.com/questions/1987694/how-to-print-the-full-numpy-array-without-truncation

In [328]:
# Pre-defined function by me for pandas data handling
def unique_value_list(df, var):
    u_list = df[var].unique()
    u_list = sorted(u_list)
    return u_list 

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

def unique_value_distribution(df,var_name): 
    category_distribution = pd.DataFrame(df[var_name].value_counts())
    print_full(category_distribution)

def df_add_var_dummy(df, var):
    df = pd.concat([df, pd.get_dummies(df[var])],axis=1)
    return df 

def select_by_var_value(df, var, value):
    sub_df = df[df[var] == value]
    return sub_df

def sum_df(df):
    print('type: ', type(df))
    print('shape: ', df.shape)

def column_name(df):
    print(len(df.columns.tolist()))
    print(df.columns.tolist())

def count(df):
    return df.count()

def head(df, num=1):
    print('shape: ', df.shape)
    return df.head(num)

def tail(df, num=1):
    print('shape: ',df.shape)
    return df.tail(num)

def re_index(df):
    df = df.reset_index(drop=True)
    return df

# load pandas dataframe or save 
# df.to_csv('df.txt', header = True)
# df = pd.read_csv('df.csv', index_col=[0])

In [ ]:
# pandas dataframe to numpy array
# df.values
# Ref: https://stackoverflow.com/questions/13187778/convert-pandas-dataframe-to-numpy-array

---
# Loading Raw Data (BLP, 1995)
* I relies on replication of BLP with following on-line note: https://kohei-kawaguchi.github.io/EmpiricalIO/assignment4.html, R and Python base)
* Ref: Data: cardata.txt (https://kohei-kawaguchi.github.io/EmpiricalIO/assignment4.html, data source)
---

In [329]:
df = pd.read_csv('cardata.txt', sep='\t')# index_col=[0]
df.columns = ["car_id", "year", "hpwt","size","air","mpd","price","share","firm_id"]
head(df,3)

shape:  (2217, 9)


,car_id,year,hpwt,size,air,mpd,price,share,firm_id
0,AD100L,71,0.487908,1.2627,0,1.982720,8.876543,0.000281,7
1,ADSUPE,71,0.465766,1.1136,0,2.201909,7.641975,0.000038,7
2,AMAMBS,71,0.452489,1.6458,0,1.504286,8.928395,0.000442,15


In [331]:
df.describe()

,year,hpwt,size,air,mpd,price,share,firm_id
count,2217.000000,2217.000000,2217.000000,2217.000000,2217.000000,2217.000000,2.217000e+03,2217.000000
mean,81.540370,0.394375,1.310159,0.241768,2.084870,11.761420,9.732479e-04,13.743798
std,5.740816,0.096643,0.237637,0.428251,0.698056,8.643777,1.144919e-03,6.259090
min,71.000000,0.170455,0.756000,0.000000,0.846069,3.393267,7.000000e-07,1.000000
25%,77.000000,0.336585,1.131279,0.000000,1.557015,6.713755,1.863000e-04,8.000000
50%,82.000000,0.375049,1.269827,0.000000,2.010417,8.728650,5.698000e-04,16.000000
75%,87.000000,0.427509,1.452700,0.000000,2.482848,13.074074,1.285500e-03,19.000000
max,90.000000,0.947581,1.888000,1.000000,6.436827,68.596774,9.472800e-03,26.000000


---
# <font color = blue> Table 3. OLS and 2SLS with IVs
---

---
### OLS
---
* Using a statsmodel, the values is slightly different; however, the overall it makes sense
* Without considering endogenous variable (=price)
* $\delta_{jt} = \ln(s_{jt}) - \ln(s_{0t})$ , where j = product, t = year
---

In [332]:
df_share = df.groupby('year', as_index=False)['share'].sum()
df_share['s0'] = 1-year_share.share
df_share['log_s0'] = np.log(df_share.s0)
print(df_share.columns)
df_share.head(3)

Index(['year', 'share', 's0', 'log_s0'], dtype='object')


,year,share,s0,log_s0
0,71,0.119894,0.880107,-0.127712
1,72,0.128604,0.871395,-0.137659
2,73,0.114224,0.885776,-0.121291


In [333]:
df = pd.merge(df, df_share, how='left', on = 'year')
df = df.rename(columns={'share_x': 'share', 'share_y': 'share_year'})
df['log_share_jt'] = np.log(df.share)
df['log_dif'] = df['log_share_jt'] - df['log_s0']
head(df,3)

shape:  (2217, 14)


,car_id,year,hpwt,size,air,mpd,price,share,firm_id,share_year,s0,log_s0,log_share_jt,log_dif
0,AD100L,71,0.487908,1.2627,0,1.982720,8.876543,0.000281,7,0.119894,0.880107,-0.127712,-8.177156,-8.049444
1,ADSUPE,71,0.465766,1.1136,0,2.201909,7.641975,0.000038,7,0.119894,0.880107,-0.127712,-10.188507,-10.060794
2,AMAMBS,71,0.452489,1.6458,0,1.504286,8.928395,0.000442,15,0.119894,0.880107,-0.127712,-7.723296,-7.595584


In [334]:
import statsmodels.formula.api as sm
ols = sm.ols(formula="log_dif ~ hpwt + air + mpd + size + price", data=df).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                log_dif   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     279.2
Date:                Fri, 28 Feb 2020   Prob (F-statistic):          6.52e-232
Time:                        20:28:27   Log-Likelihood:                -3319.3
No. Observations:                2217   AIC:                             6651.
Df Residuals:                    2211   BIC:                             6685.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.0716      0.253    -39.823      0.0

---
# Interpretation
* constant, mpd, size, price are significant at 1% level
* price coefficient is negative (= law of demand, normal good)
* R squre is 0.386 so that the variation of Y (=log(s_jt) - log(s0)) is 61.4% explained by unobserved terms (=need to consider undobserved charateristics) 
---

---
### 2SLS
---
* Personally, I think using R or Stata may better for econometric part such as conventional methods rather than Python for sure.
* Just changing kernel from python to stata or R, it is okay in jupter notebook; however, this is a homework so as it requires I am usin python only. 
* Ref: https://python.quantecon.org/ols.html (2sls)
* Package: https://pypi.org/project/linearmodels/  (llinearmodels, extened version of statmodels for IV2SLS)
* Package: https://github.com/bashtage/linearmodels/blob/master/examples/iv_basic-examples.ipynb (IV2SLS, example)
---

#### Optimal IVs for 2SLS
---
* (5.8) Optimal IVs (paper, 861 page)
* $ z_{jtk}, \sum_{r\neq j, r\in F_{f}} z_{rtk} , \sum_{r\neq  j, r\notin F_{f}} z_{rtk}$ 
* $F_{f}$ : is the set of products produced by firm f  
* The second term implies the sum of kth product caharteristics own firm product exlcuding product j 
* The thrid term implies the sum of kth product caharteristics other firm product exlcuding product j
* The number of own firm products, the nubmer of rival-firm products become instruments (in paper)
---

In [335]:
df_iv_year_firm = df.groupby(['year','firm_id'], as_index=False)[ 'hpwt', 'air',  'mpd',  'size'].sum()
df_iv_year_firm['own_count'] = df.groupby(['year','firm_id'], as_index=False).car_id.count()['car_id']
df_iv_year_firm  = df_iv_year_firm.rename(columns={'hpwt': 'sum_hpwt_tf', 'air':'sum_air_tf', 'mpd':'sum_mpd_tf', 'size':'sum_size_tf'})
df_iv_year_firm.head(3)

,year,firm_id,sum_hpwt_tf,sum_air_tf,sum_mpd_tf,sum_size_tf,own_count
0,71,1,2.230527,0,11.381168,5.3569,5
1,71,2,0.867832,0,5.267225,1.8980,2
2,71,3,0.243750,0,2.836113,0.8586,1


In [336]:
df = pd.merge(df, df_iv_year_firm, how='inner', on =['year', 'firm_id'])
df.head(3)

,car_id,year,hpwt,size,air,mpd,price,share,firm_id,share_year,s0,log_s0,log_share_jt,log_dif,sum_hpwt_tf,sum_air_tf,sum_mpd_tf,sum_size_tf,own_count
0,AD100L,71,0.487908,1.2627,0,1.982720,8.876543,0.000281,7,0.119894,0.880107,-0.127712,-8.177156,-8.049444,1.841746,0,11.417885,5.6416,5
1,ADSUPE,71,0.465766,1.1136,0,2.201909,7.641975,0.000038,7,0.119894,0.880107,-0.127712,-10.188507,-10.060794,1.841746,0,11.417885,5.6416,5
2,VW411,71,0.328294,1.1946,0,2.060604,7.404938,0.000372,7,0.119894,0.880107,-0.127712,-7.897693,-7.769980,1.841746,0,11.417885,5.6416,5


In [337]:
df_iv_year = df.groupby(['year'], as_index=False)[ 'hpwt', 'air',  'mpd',  'size'].sum()
df_iv_year['year_all_count'] = df.groupby(['year'], as_index=False).car_id.count()['car_id']
df_iv_year = df_iv_year.rename(columns={'hpwt': 'sum_hpwt_t', 'air':'sum_air_t', 'mpd':'sum_mpd_t', 'size':'sum_size_t'})
df_iv_year.head(5)

,year,sum_hpwt_t,sum_air_t,sum_mpd_t,sum_size_t,year_all_count
0,71,46.925503,0,176.058173,132.7013,92
1,72,36.227952,4,168.302316,129.7578,89
2,73,33.112552,6,159.925826,126.1010,86
3,74,26.496995,9,108.976167,102.6927,72
4,75,32.959320,10,143.561553,131.7138,93


In [338]:
df = pd.merge(df, df_iv_year, how='inner', on =['year'])

In [339]:
df.head(3)

,car_id,year,hpwt,size,air,mpd,price,share,firm_id,share_year,...,sum_hpwt_tf,sum_air_tf,sum_mpd_tf,sum_size_tf,own_count,sum_hpwt_t,sum_air_t,sum_mpd_t,sum_size_t,year_all_count
0,AD100L,71,0.487908,1.2627,0,1.982720,8.876543,0.000281,7,0.119894,...,1.841746,0,11.417885,5.6416,5,46.925503,0,176.058173,132.7013,92
1,ADSUPE,71,0.465766,1.1136,0,2.201909,7.641975,0.000038,7,0.119894,...,1.841746,0,11.417885,5.6416,5,46.925503,0,176.058173,132.7013,92
2,VW411,71,0.328294,1.1946,0,2.060604,7.404938,0.000372,7,0.119894,...,1.841746,0,11.417885,5.6416,5,46.925503,0,176.058173,132.7013,92


In [340]:
df['own_hpwt'] = df.sum_hpwt_tf - df.hpwt
df['own_size'] = df.sum_size_tf - df.size
df['own_air'] = df.sum_air_tf - df.air
df['own_mpd'] = df.sum_mpd_tf - df.mpd

In [341]:
df['rival_hpwt'] = df.sum_hpwt_t - df.hpwt -df.own_hpwt
df['rival_size'] = df.sum_size_t - df.size -df.own_size
df['rival_air'] = df.sum_air_t - df.air - df.own_air
df['rival_mpd'] = df.sum_mpd_t - df.mpd - df.own_mpd

In [342]:
df['rival_count'] = df.year_all_count - df.own_count

In [343]:
df_iv = df[['car_id', 'year', 'firm_id', 'share_year', 's0', 'log_s0', 'log_share_jt', 'log_dif', 'hpwt', 'size', 'air', 'mpd', 'price', 'share', 'own_hpwt', 'own_size', 'own_air', 'own_mpd', 'rival_hpwt', 'rival_size', 'rival_air', 'rival_mpd', 'own_count', 'rival_count']]  

In [344]:
df_iv['cons'] = 1

/opt/apps/anaconda3/19.10.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [345]:
head(df_iv,1)

shape:  (2217, 25)


,car_id,year,firm_id,share_year,s0,log_s0,log_share_jt,log_dif,hpwt,size,...,own_size,own_air,own_mpd,rival_hpwt,rival_size,rival_air,rival_mpd,own_count,rival_count,cons
0,AD100L,71,7,0.119894,0.880107,-0.127712,-8.177156,-8.049444,0.487908,1.2627,...,-55419.3584,0,9.435165,45.083756,-8740.9403,0,164.640289,5,87,1


In [346]:
df_iv.columns

Index(['car_id', 'year', 'firm_id', 'share_year', 's0', 'log_s0',
       'log_share_jt', 'log_dif', 'hpwt', 'size', 'air', 'mpd', 'price',
       'share', 'own_hpwt', 'own_size', 'own_air', 'own_mpd', 'rival_hpwt',
       'rival_size', 'rival_air', 'rival_mpd', 'own_count', 'rival_count',
       'cons'],
      dtype='object')

In [347]:
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from linearmodels.iv import IV2SLS # https://pypi.org/project/linearmodels/ 
# (install) pip install --user linearmodels

In [348]:
iv = IV2SLS(dependent=df_iv['log_dif'],
            exog =  df_iv[['cons', 'hpwt', 'air', 'mpd' , 'size']],
            endog= df_iv['price'],
            instruments=df_iv[['own_hpwt', 'own_size', 'own_air', 'own_mpd',
                               'rival_hpwt', 'rival_size', 'rival_air', 'rival_mpd', 
                               'own_count', 'rival_count']]).fit(cov_type='unadjusted') # 14 IVs, it is less than paper, pyblp supports 19 ivs
print(iv.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                log_dif   R-squared:                      0.3492
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3477
No. Observations:                2217   F-statistic:                    1020.0
Date:                Fri, Feb 28 2020   P-value (F-stat)                0.0000
Time:                        20:28:59   Distribution:                  chi2(5)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
cons          -9.9153     0.2623    -37.796     0.0000     -10.429     -9.4011
hpwt           1.2260     0.4031     3.0414     0.00

---
# Interpretation
---
* There are limitation using Python for 2sls. 
* IVs is not 19s like Pyblp, I used 14s and it is based on Homoskedastic assumption in 2SLS in over-identification case.
* It is obvious it is not Homoskedastic and over-identification so that using GMM Estimator will be better.
* However, anyway, the coefficient of price is much higher in the case of considering endogendity in price variable.
---

---
# <font color = blue> Table 4. Estimated Parameters of tmhe Demand and Pricing Equations
* DF: df_iv
* IVs:  'hpwt', 'air', 'mpd' , 'size', 'own_hpwt', 'own_size', 'own_air', 'own_mpd','rival_hpwt', 'rival_size', 'rival_air', 'rival_mpd', 'own_count', 'rival_count'
* True Market Share: 'share'
---

#### GMM Estimation
* $ \delta_{jt} = x_{jt}\beta + \xi_{jt}$

In [398]:
ns = 50000
nrow = 2217
tol = 1e-6

In [399]:
X = df_iv[["cons", "hpwt", "air", "mpd", "size"]]
X = np.asarray(X)
print(X.shape)
print('Covariate number: ', X.shape[1]) # column number = 5

(2217, 5)
Covariate number:  5


In [400]:
Z = df_iv[["cons", "hpwt", "air", "mpd", "size", "own_count", "own_hpwt", "own_air", "own_mpd", "own_size", "rival_count", "rival_hpwt", "rival_air", "rival_mpd", "rival_size"]]
print('IV dimneion including covariate exlcuding price :', Z.shape)
Z = np.asarray(Z)

IV dimneion including covariate exlcuding price : (2217, 15)


In [410]:
# 2SLS projection matrix (=not coefficent) beta_2sls = p_delta @ delta 
# @ matrix innuer product in python 
# weight = Z.T @ Z : (15, 15) Postive definite sysmetric matrix
weight = np.dot(Z.T,Z)
projection_matrix_delta  = inv(X.T @ Z @ inv(weight) @ Z.T @ X) @ X.T @ Z @ inv(weight) @ Z.T
projection_matrix_delta.shape

(5, 2217)

In [402]:
# year from 71-90: 20 years
print('number of years: ', df_iv.year.nunique())
n_years = df_iv.year.nunique()

number of years:  20


In [403]:
Y = np.asarray(pd.get_dummies(df_iv["year"]), dtype = np.float32)
print('year dummy: ', Y.shape)
YY_T =  np.dot(Y,Y.T)
print('year * year_T shape : ', YY_T.shape)

year dummy:  (2217, 20)
year * year_T shape :  (2217, 2217)


In [404]:
np.random.seed(11)
v = np.random.randn(ns, X.shape[1]) # ns = number of simulation = 50,000
print('50,000 draws in Xs format :', v.shape)

50,000 draws in Xs format : (50000, 5)


In [405]:
np.random.seed(12)

year_mean = np.array([2.01156, 2.06526, 2.07843, 2.05775, 2.02915, 2.05346, 2.06745, 2.09805, 2.10404,2.07208, 
                   2.06019, 2.06561, 2.07672, 2.10437, 2.12608, 2.16426, 2.18071, 2.18856, 2.21250, 2.18377]).reshape(20,1)
y_sigma = 1.72

In [406]:
np.random.seed(18)

ar = np.random.randn(n_years,ns) # (20, 50,000)

y = Y @ np.exp(year_mean + y_sigma*ar) # mean + y_sigma * ar
y.shape

(2217, 50000)

In [408]:
s_true = np.asarray(df_iv["share"]).reshape(nrow,1)
s_true.shape

(2217, 1)

In [409]:
price = np.asarray(df_iv["price"]).reshape(nrow,1)
price.shape

(2217, 1)

In [360]:
# real price = p/y
real_price = np.dot(price,np.ones((1,ns)))/y
real_price.shape

(2217, 50000)

In [362]:
global deltam
deltam = np.exp(df_iv["log_dif"]) # log s_jk -  log s0
deltam = np.asarray(deltam).reshape(nrow,1)
deltam.shape

(2217, 1)

---
### Estimated Market Share
* Probability (product j being sold in year t) = $\approx \frac{1}{ns}\sum_{i=1}^{ns}\frac{exp(\delta_{jt}+\mu_{ijt})}{1+\sum_{k=1}^{J_t} \exp(\delta_{kt}+\mu_{ikt})}$
* $ \delta_{jt} = x_{jt}\beta + \xi_{jt}$
* $ \mu_{ijt} = -\alpha \frac{p_{jt}}{y_{it}} + \sum_{k=1}^{K}\sigma_k v_{ik} x_{jkt}$, where the real price concept is from BLP(1999)
* $ u_{ijt} = \delta_{jt} + \mu_{ijt} + \epsilon_{ijt} $ 
---

---
### Market share prediction
---

In [411]:
# This function used for contraction mapping

def s_hat(exp_delta0, exp_mu):
    numerator   = exp_delta0 @ np.ones((1, ns)) * exp_mu       # exp_delta0 (2217, 1) x (1, 50,000) x (50,000, )
    denominator = np.dot(Y, np.dot(Y.T,numerator))           # Y = year dummy = (2217, 20) x Y.T (=20, 2217) x (2217)
    fraction    = numerator/(1+denominator)
    s_estimated = (1/ns)*(fraction.sum(1))
    return(s_estimated)

---
### Contraction Mapping:
* As a result : $\delta$ converge and multiply by projection matrx of 2SLS
---
* $ \delta^{n+1} = \delta^{n}  + \log(s^{estimated}) - \log(s(\theta)) $ : until converge, criteria of convergence is tol
* $ \theta = (\beta, \alpha, \sigma)$  : parameter to be estimated
* $ \sigma = (\sigma_0,..., \sigma_k)$ : coefficient with attribute of product and consumer preference
---
* $ \delta_{jt} = x_{jt}\beta + \xi_{jt}$
* $ \mu_{ijt} = -\alpha \frac{p_{jt}}{y_{it}} + \sum_{k=1}^{K}\sigma_k v_{ik} x_{jkt}$, where the real price concept is from BLP(1999)
* $ u_{ijt} = \delta_{jt} + \mu_{ijt} + \epsilon_{ijt} $ 
---



In [412]:
ns = 50000
nrow = 2217
tol = 1e-6

In [413]:
# input : X, output : delta
def contraction_mapping(sigma, alpha):
    
    exp_delta0 = deltam # from global variable
    
    draws = v * np.kron(np.ones((ns,1)), sigma) # v = (50000, 5)
    
    # Kronecker product: https://en.wikipedia.org/wiki/Kronecker_product
    # np.kron : https://docs.scipy.org/doc/numpy/reference/generated/numpy.kron.html    
    
    exp_mu = (- real_price * alpha + np.exp(np.dot(X, draws.T) )) # real_price (2217, 50000) * alpha + exp(X x draws')
    
    difference = 1
    iteration = 0
    
    while(difference > tol and iteration < 1000):
        
        ratio = s_true/s_hat(exp_delta0, exp_mu)
        ratio.reshape(nrow,1)
        
        exp_delta1 = exp_delta0 * ratio # interation part exp(theta1) = exp(theta0)*exp(log(true_s) - log(s_hat)) = exp(theta0)*(true_s/s_hat)
        
        difference = (abs(exp_delta1/exp_delta0-1)).max()
 
        exp_delta0 = exp_delta1
        iteration += 1
        
    return(np.log(exp_delta0)) # np.log(exp(theta0) =  theta come out

### GMM

In [414]:
def gmm_criteria(params):
    
    sigma = params[0:5]
    alpha = params[5]
    
    delta = contraction_mapping(sigma,alpha)
    
    beta = np.dot(projection_matrix_delta,delta)
    deltam = delta
    
    xi = delta - np.dot(X,beta)
    g = (1/nrow)* (np.dot(Z.T,xi)) # moment condition
    
    object_value = np.dot(g.T,g)
    object_value = object_value[0,0]
    return(object_value)

In [415]:
start_point = np.array([3.612, 4.628, 1.818, 1.050, 2.056, 43.501])

In [416]:
gmm_min = minimize(gmm_criteria, start_point, method="L-BFGS-B", tol = 1e-2, options={'disp': True})
print(gmm_min)

ValueError: cannot reshape array of size 4915089 into shape (2217,1)

---
# Until Here.
* I think share prediction part is wired. So, in the future, it may better for me to look more closely verified m-file and code such as Nevo blp.
* The process is not yet well understood by me.
* Working with matlab first, then moving to python may better.
* 2sls and data modification requires more time, so using R kernel combining with jupyterlab may more efficient.
* I just modify the code in python way, therefore, MC intergration part is not well defined and understood.
* GMM has a two-step approach. Implementation of GMM may be a good practice before doing GMM in here.
* It is a good practice to know better understanding of discrete choice; however, due to limited ability and my load on independent dissertation. I finished in here.
* All the falut are mine and most of contribution from web-resoure, I just modifed and analyzed it in pythonic way.
* Thank you.
---

In [ ]:
# no run
print("sigma: ", abs(gmm_min.x[0:5]),"\n", "alpha: ", gmm_min.x[5],sep="")
print("coefficient:", (projection_matrix_delta @ contraction_mapping(gmm_min.x[0:5], gmm_min.x[5])).T[0])